In [1]:
# mlrun: start-code

In [2]:
import sys
import pickle
import pandas as pd
from typing import Tuple

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

from mlrun import get_or_create_ctx

In [3]:
def get_data(test_size:float=0.3):
    """
    Load test iris dataset. Split into X_train, X_test, y_train, y_test.
    
    :param test_size: Percentage of dataset to use for test set.
    
    :returns:         X_train, X_test, y_train, y_test
    """
    X, y = load_iris(return_X_y=True, as_frame=True)
    return train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [4]:
def str_to_class(str:str):
    """
    Turns a string into Python class. Used to dynamically load Sklearn model classes.
    Note: Desired class must be imported at top of script.
    
    :param str: String to dynamically load as Python class.
    
    :returns:   Python class corresponding to string.
    """
    return eval(str)

In [5]:
def build_model(model_class:str, model_params:dict):
    """
    Build Sklearn model of certain type with parameters.
    
    :param model_class:  Sklearn class to create.
    :param model_params: Dict of model parameters to use.
    
    :returns:            Newly built sklearn model.
    """
    model_class = str_to_class(model_class)
    return model_class(**model_params)

In [6]:
def train_model(
    model,
    hyperparameters:dict,
    X_train: pd.DataFrame,
    y_train: pd.Series
):
    """
    Train Sklearn model with random hyperparameter search.
    
    :param model:           Sklearn model to train.
    :param hyperparameters: Hyperparameter grid to search.
    :param X_train:         Training data.
    :param y_train:         Training labels.
    
    :returns:               Best trained Sklearn model.
    """
    clf = RandomizedSearchCV(model, hyperparameters, random_state=0)
    search = clf.fit(X_train, y_train)
    return search.best_estimator_

In [7]:
def evaluate_model(model, X_test: pd.DataFrame, y_test: pd.Series) -> dict:
    """
    Evaluates trained SKlearn model with common metrics.
    
    :param model:  Trained Sklearn model.
    :param X_test: Test data to evaluate with.
    
    :returns:      Dict of evaluation metrics.
    """
    y_pred = model.predict(X_test)
    return {
        "accuracy" : accuracy_score(y_test, y_pred),
        "f1" : f1_score(y_test, y_pred, average="micro"),
        "precision" : precision_score(y_test, y_pred, average="micro"),
        "recall" : recall_score(y_test, y_pred, average="micro"),
    }

In [8]:
def main(model_config: dict):
    """
    Main training function. Loads data, trains models using specified
    classes/parameters/hyperparameters, evaluates models, and exports
    models to disk.
    
    :param model_config: Dict of model classes and corresponding parameters
                         and hyperparameters to use while training.
    """
    # Get MLRun context
    with get_or_create_ctx("train") as context:
    
        # Get datasets
        context.logger.info("Getting data")
        X_train, X_test, y_train, y_test = get_data()
        
        # Log datasets
        context.logger.info("Logging datasets")
        context.log_dataset(key="X_train", df=X_train, format="csv", artifact_path=context.artifact_path)
        context.log_dataset(key="X_test", df=X_test, format="csv", artifact_path=context.artifact_path)
        context.log_dataset(key="y_train", df=y_train.to_frame(), format="csv", artifact_path=context.artifact_path)
        context.log_dataset(key="y_test", df=y_test.to_frame(), format="csv", artifact_path=context.artifact_path)

        # For all models in config
        for name, config in model_config.items():

            # Build base model
            context.logger.info(f"Building: {name}")
            model = build_model(model_class=name, model_params=config["params"])

            # Train model with hyperparameter tuning
            context.logger.info(f"Training: {name}")
            model = train_model(model, config["hyperparameters"], X_train, y_train)
            context.logger.info(f"Best parameters: {model.get_params()}")

            # Evaluate model
            context.logger.info(f"Evaluating: {name}")
            metrics = evaluate_model(model, X_test, y_test)
            context.logger.info(f"Evaluation metrics: {metrics}")

            # Export model to disk
            context.logger.info(f"Saving: {name}")
            pickle.dump(model, open(f"{name}.pkl", 'wb'))
            
            # Log model
            context.logger.info("Logging model")
            context.log_model(
                key=f"{name}_model",
                artifact_path=context.artifact_path,
                model_file=f"{name}.pkl",
                metrics=metrics,
                parameters=model.get_params(),
                framework="sklearn",
            )

In [9]:
# mlrun: end-code

In [10]:
from mlrun import code_to_function, auto_mount
from mlrun.runtimes.utils import generate_function_image_name

In [15]:
fn = code_to_function(
    name="my-training-job-tracking",
    project="royal-cyber",
    handler="main",
    image="mlrun/mlrun",
    kind="job",
    requirements="requirements.txt" # or requirements=["pandas==1.3.2", "numpy"]
).apply(auto_mount())

In [16]:
# Build image using requirements.txt
fn.deploy(skip_deployed=False) # To skip build if image exists, use skip_deployed=True
fn.spec.image = generate_function_image_name(fn)

> 2021-08-20 22:29:30,110 [info] Started building image: .mlrun/func-royal-cyber-my-training-job-tracking:latest
INFO[0000] Retrieving image manifest mlrun/mlrun:0.6.5  
INFO[0000] Retrieving image manifest mlrun/mlrun:0.6.5  
INFO[0001] Built cross stage deps: map[]                
INFO[0001] Retrieving image manifest mlrun/mlrun:0.6.5  
INFO[0001] Retrieving image manifest mlrun/mlrun:0.6.5  
INFO[0001] Executing 0 build triggers                   
INFO[0001] Unpacking rootfs as cmd RUN python -m pip install pandas==1.3.2 requires it. 
INFO[0015] RUN python -m pip install pandas==1.3.2      
INFO[0015] Taking snapshot of full filesystem...        
INFO[0026] cmd: /bin/sh                                 
INFO[0026] args: [-c python -m pip install pandas==1.3.2] 
INFO[0026] Running: [/bin/sh -c python -m pip install pandas==1.3.2] 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.0
    Uninstalling pandas-1.3.0:
      Successfully uninstalled pandas-1.3.0
You 

In [17]:
model_config = {
    "LogisticRegression": {
        "params" : {'solver': 'saga', 'tol': 0.01, 'max_iter': 200, 'random_state': 0},
        "hyperparameters" : {"penalty": ['l2', 'l1'], "C" : [0.6, 0.8, 1.0, 1.1, 1.2]}
    },
    "RandomForestClassifier" : {
        "params" : {"max_depth": 2, "random_state": 0},
        "hyperparameters" : {"n_estimators" : [10, 50, 100, 200], "criterion": ["gini", "entropy"]}
    }
}

In [18]:
fn.run(params={"model_config" : model_config})

> 2021-08-20 22:30:55,745 [info] starting run my-training-job-tracking-main uid=32c8cd1816a14b5eb49d99ab92521b08 DB=http://mlrun-api:8080
> 2021-08-20 22:30:55,988 [info] Job is running in the background, pod: my-training-job-tracking-main-whhqj
> 2021-08-20 22:31:02,777 [info] Getting data
> 2021-08-20 22:31:02,783 [info] Logging datasets
> 2021-08-20 22:31:03,182 [info] Building: LogisticRegression
> 2021-08-20 22:31:03,182 [info] Training: LogisticRegression
> 2021-08-20 22:31:03,403 [info] Best parameters: {'C': 0.6, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 200, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 0, 'solver': 'saga', 'tol': 0.01, 'verbose': 0, 'warm_start': False}
> 2021-08-20 22:31:03,403 [info] Evaluating: LogisticRegression
> 2021-08-20 22:31:03,406 [info] Evaluation metrics: {'accuracy': 0.9555555555555556, 'f1': 0.9555555555555556, 'precision': 0.9555555555555556, 'rec

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
royal-cyber,...92521b08,0,Aug 20 22:31:02,completed,my-training-job-tracking-main,v3io_user=nickkind=jobowner=nickhost=my-training-job-tracking-main-whhqj,,"model_config={'LogisticRegression': {'params': {'solver': 'saga', 'tol': 0.01, 'max_iter': 200, 'random_state': 0}, 'hyperparameters': {'penalty': ['l2', 'l1'], 'C': [0.6, 0.8, 1.0, 1.1, 1.2]}}, 'RandomForestClassifier': {'params': {'max_depth': 2, 'random_state': 0}, 'hyperparameters': {'n_estimators': [10, 50, 100, 200], 'criterion': ['gini', 'entropy']}}}",,X_trainX_testy_trainy_testLogisticRegression_modelRandomForestClassifier_model


to track results use .show() or .logs() or in CLI: 
!mlrun get run 32c8cd1816a14b5eb49d99ab92521b08 --project royal-cyber , !mlrun logs 32c8cd1816a14b5eb49d99ab92521b08 --project royal-cyber
> 2021-08-20 22:31:11,428 [info] run executed, status=completed
